In [0]:
"https://towardsdatascience.com/random-forest-in-python-24d0893d51c0"

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import pydot
from sklearn import metrics

sns.set_style('darkgrid')


In [2]:
cen = pd.read_csv('fulfilment_center_info.csv')
mel = pd.read_csv('meal_info.csv')
main = pd.read_csv('train.csv')

inner1 =  pd.merge(left=main ,right=cen, left_on='center_id', right_on='center_id')
mark0 = pd.merge(left=inner1 ,right=mel, left_on='meal_id', right_on='meal_id')
mark0.shape

(456548, 15)

In [0]:
mark1 = mark0

In [0]:
mark1.drop(['id','center_id','meal_id','base_price','city_code','region_code','week'], axis=1,inplace=True)

In [5]:
mark1['emailer_for_promotion'] = mark1['emailer_for_promotion'].replace([0,1],['email_no','email_yes'])
mark1['homepage_featured'] = mark1['homepage_featured'].replace([0,1],['home_no','home_yes'])

hot = mark1[['emailer_for_promotion','homepage_featured','center_type','category','cuisine']]
dum = pd.get_dummies(hot,drop_first=True)

mark1.drop(['emailer_for_promotion','homepage_featured','center_type','category','cuisine'],axis=1,inplace=True)

mark1 = pd.concat([mark1, dum], axis=1)
mark1.head(1)

,checkout_price,num_orders,op_area,emailer_for_promotion_email_yes,homepage_featured_home_yes,center_type_TYPE_B,center_type_TYPE_C,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Indian,cuisine_Italian,cuisine_Thai
0,136.83,177,2.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
foxtrot = mark1

In [0]:
X_train, X_test, y_train, y_test = train_test_split(foxtrot.drop('num_orders',axis=1), 
                                                    foxtrot['num_orders'], test_size=0.30, 
                                                    random_state=101)

In [0]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf.fit(X_train, y_train)
# Predictions
predictions = rf.predict(X_test)

In [0]:
feature_list = list(X_train.columns)

In [0]:
feature_list = list(X_train.columns)

# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

In [0]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(X_train, y_train)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png')

In [15]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: checkout_price       Importance: 0.3669
Variable: op_area              Importance: 0.1955
Variable: category_Rice Bowl   Importance: 0.0974
Variable: homepage_featured_home_yes Importance: 0.096
Variable: category_Sandwich    Importance: 0.0561
Variable: emailer_for_promotion_email_yes Importance: 0.0514
Variable: cuisine_Italian      Importance: 0.0273
Variable: center_type_TYPE_C   Importance: 0.0217
Variable: center_type_TYPE_B   Importance: 0.0177
Variable: cuisine_Indian       Importance: 0.016
Variable: category_Salad       Importance: 0.014
Variable: cuisine_Thai         Importance: 0.0133
Variable: category_Extras      Importance: 0.0107
Variable: category_Pasta       Importance: 0.0068
Variable: category_Pizza       Importance: 0.0045
Variable: category_Soup        Importance: 0.002
Variable: category_Other Snacks Importance: 0.0011
Variable: category_Seafood     Importance: 0.0005
Variable: category_Biryani     Importance: 0.0004
Variable: category_Starters    Impor

In [16]:
feature = pd.DataFrame(feature_importances, columns=['Varialble','Importance'])
feature

,Varialble,Importance
0,checkout_price,0.3669
1,op_area,0.1955
2,category_Rice Bowl,0.0974
3,homepage_featured_home_yes,0.0960
4,category_Sandwich,0.0561
5,emailer_for_promotion_email_yes,0.0514
6,cuisine_Italian,0.0273
7,center_type_TYPE_C,0.0217
8,center_type_TYPE_B,0.0177
9,cuisine_Indian,0.0160


In [0]:
feature.to_excel('feature_imp.xlsx')